In [ ]:
import pandas as pd 
import numpy as np

from  statsmodels.tsa.stattools import acf 
import matplotlib.pyplot as plt

"""

Scikit-learn

"""
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,RidgeClassifier,RidgeClassifierCV
from sklearn.ensemble  import AdaBoostClassifier, GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB,CategoricalNB
from xgboost import XGBClassifier

from sklearn.linear_model import ElasticNet
from lightgbm import LGBMClassifier

def strategy_long_only(Capital, Qty, Vix, Pred):
    
    if Qty != 0 and Pred == 0:
        Capital = Qty * Vix
        Qty = 0
    if Qty == 0 and Pred == 1:
        Qty = Capital / Vix
        Capital = 0
    return Capital,Qty

def strat_long_short(Capital, Qty, Vix, Proba, Short):

    if proba>0.55 and Capital!= 0 :
        if Short == True:
            Capital -= Qty * Vix
        Qty = Capital/Vix
        Capital = 0
        Short = False
    
    elif proba< 0.45 and Short== False:
        if Qty !=0:
            Capital = Qty*Vix

        Qty = Capital / Vix
        Capital += Qty*Vix
        Short = True
    return Capital,Qty, Short

def strategy_long_only_SP500(Capital, Qty, SP500, Proba):
    
    if Qty != 0 and Proba > 0.55:
        Capital = Qty * SP500
        Qty = 0
    if Qty == 0 and Proba < 0.45:
        Qty = Capital / SP500
        
    return Capital,Qty

data = pd.read_csv(r"df_merged_data_hurst.csv",sep = ';', decimal = ',')

sp500 = data[["Date","SP500"]].copy()

data["SP500"] = data["SP500"].pct_change()

data = data.set_index("Date")

list_name_t1 = [i+"_t_1" for i in data.columns]
list_name_t2 = [i+"_t_2" for i in data.columns]

df_t1 = data.shift()
df_t2 = data.shift(2)
data["VIX_t_3"] = data["VIX"].shift(3)
data["10Y"] = data["10Y"].diff()
data["3M"] = data["3M"].diff()

df_t1.columns = list_name_t1 
df_t2.columns = list_name_t2


n_data = pd.concat([data,df_t1,df_t2], axis = 1 ).dropna()

n_data["Y_pred"]     = 1*(n_data["VIX"] > n_data["VIX_t_1"]) 
n_data["Y_t_1_pred"] = 1*(n_data["VIX_t_1"] > n_data["VIX_t_2"]) 
n_data["Y_t_2_pred"] = 1*(n_data["VIX_t_2"] > n_data["VIX_t_3"]) 

del n_data["VIX_t_3"]

#n_data.to_csv("Dataset.csv")

X_all = n_data.loc[:, ~n_data.columns.isin(['Y_pred',"VIX"])]

Y_all = n_data["Y_pred"]

X_train, X_test, y_train, y_test = train_test_split(X_all, Y_all, test_size=0.1, random_state=1, shuffle = False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, shuffle = False) # 0.25 x 0.8 = 0.2

print(X_train.shape,  X_val.shape,X_test.shape, y_train.shape, y_val.shape,y_test.shape)

models = [LogisticRegression,LinearDiscriminantAnalysis,BernoulliNB, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,
KNeighborsClassifier] #pas de rpart ni bglm car pas de package sous Python (uniquement sous R, et pas le temps de recoder)

#          ('EN_AUC',LogisticRegression(penalty='elasticnet',solver='saga')),
#           ('EN_Class',LogisticRegression(penalty='elasticnet',solver='saga')),
#           ('EN_Dev',LogisticRegression(penalty='elasticnet',solver='saga')),
#           ('LDA',LinearDiscriminantAnalysis()),
#           ('RDA',LinearDiscriminantAnalysis(solver='eigen',shrinkage='auto')),
#Random Forest,B_GLM
models = [
    ('RIDGE_AUC',LogisticRegression()),
          ('RIDGE_Class',LogisticRegression()),
          ('RIDGE_Dev',LogisticRegression()),
       ('LASSO_AUC',LogisticRegression(penalty='l1')),
    ("LIGHTGBM",LGBMClassifier()),
          ('LASSO_Class',LogisticRegression(penalty='l1')),
          ('LASSO_Dev',LogisticRegression(penalty='l1')),
          ('Naive_B',BernoulliNB()),
          ('Bagging_Classfer',BaggingClassifier()),
          ('Random_Frst',RandomForestClassifier()),
          ('adaboost',AdaBoostClassifier(base_estimator=DecisionTreeClassifier())),
          ('GradientB_class',GradientBoostingClassifier()),
          ('KNearest_Neigh',KNeighborsClassifier()),
          ("xgboost",  XGBClassifier(min_child_weight=0,\
                       gamma=0, subsample=0.7,\
                       colsample_bytree=1.0,\
                       objective='reg:squarederror', nthread=-1,\
                       scale_pos_weight=1, seed=27,\
                       reg_alpha=0.00006))
         ]

grids = {
    'RIDGE_AUC':{'solver':['newton-cg','lbfgs','liblinear','sag','saga']},
         'RIDGE_Class':{'solver':['newton-cg','lbfgs','liblinear','sag','saga']},
         'RIDGE_Dev':{'solver':['newton-cg','lbfgs','liblinear','sag','saga']},
        'LASSO_AUC':{'solver':['liblinear','saga']},
            "LIGHTGBM":{"boosting_type " : ["gbdt","dart"],
                       "num_leaves": [100,150],
                        "learning_rate " : [0.01,0.2,0.3]
                       },
         'LASSO_Class':{'solver':['liblinear','saga']},
         'LASSO_Dev':{'solver':['liblinear','saga']},
         'Naive_B':{'alpha':[0.2,0.4,0.6,0.8]},
         'Bagging_Classfer':{'n_estimators':[10,100,1000]},
         'Random_Frst':{'n_estimators':[5,20,50,100],
                         'max_depth':[5,7,10,12,15],
                         'min_samples_split':[2,4,6],
                         'min_samples_leaf':[1,2,3,4,5],
                         'max_features':['sqrt', 'log2'],
                        },
         'adaboost' : {'n_estimators': [100,300,500],
               'learning_rate' :[0.1,0.5,1.0],
               'base_estimator__max_depth':[i for i in range(2,7,2)], #Pour atteindre les paramètres du WL
               'base_estimator__min_samples_leaf' :[0.01]},
         'KNearest_Neigh':{'leaf_size':list(range(1,50)),
                           'n_neighbors':list(range(1,30)),
                           'p':[1,2]},
         'GradientB_class':{'learning_rate': [0.01,0.05,0.1],
                  'subsample'    : [1.0,0.9, 0.8],
                  'n_estimators' : [300,500],      
                  'max_depth'    : [3,5]},
         'xgboost' : { 'max_depth': [2,4, 6],
                        'n_estimators': [1000, 3000, 5000],
                        'learning_rate': [0.09, 0.1,0.11]}
        }

sp500.set_index("Date",inplace=True)

import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

def calculate_dev(y_true,y_pred):
    return (2*(y_true * np.log(y_true/y_pred) - (y_true-y_pred))).sum()

columns_dic = ["Proba","Pred","Cap1","Cap2","Cap3","Qty1","Qty2","Qty3","Short"]
modelOpt_list = []

dico_model = {}

dat = X_train.iloc[-1,:].name
dict_s = {"Proba" : 0 ,"Pred" :0,"Cap1" : 100,"Cap2": 100,"Cap3":100,"Qty1":0,"Qty2":0,"Qty3":0, "Short" : False}

for name, model in models:
    dico_model[name] = pd.DataFrame(index = [dat],data = dict_s)
for i,p in enumerate(range(len(X_val))):
    for name, model in models:
        
        #Vix = data.iloc[p,1]
        start_time=time.time()
        #pipe = Pipeline(steps=[('preprocessor', preproc), (name, model)])
          
        
        # Elastic Net 
        
        regr = ElasticNet(random_state=0,alpha = 0.50)
        
        regr.fit(X_train, y_train)

        #print("Elastic Net picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

        coef = pd.Series(regr.coef_, index = X_train.columns)

        X_train_temp = X_train[coef[coef!=0].index.to_list()]
        
        if "AUC" in name : 
            clf = GridSearchCV(model,grids[name],scoring='roc_auc', cv=3,n_jobs=-1)
        if "Class" in name : 
            clf = GridSearchCV(model, grids[name],scoring='CLASS', cv=3,n_jobs=-1)
        if "Dev" in name : 
            clf = GridSearchCV(model, grids[name],scoring='DEV', cv=3,n_jobs=-1)
            
        clf = GridSearchCV( model,grids[name],scoring="accuracy", cv=3,n_jobs=-1)
        clf.fit(X_train_temp, y_train)
        modelOpt_list.append((name,clf))
        
        
        print('\033[1m' + " Results for {}:".format(name) + '\033[0m')
        print(' Returned hyperparameter: {}'.format(clf.best_params_))
        print(' Best classification score in validation set is: {}'.format(round(clf.best_score_,3)*100))
        #print(' classification score on test is: {}'.format(- clf.score(X_test, y_test)*100),end='\n\n')
        print("--------- %s secondes ---------" % (round(time.time() - start_time,2)))
        print('-----------------------------------------------------')
        

        
        X_val_temp = X_val[coef[coef!=0].index.to_list()]
        # Prédiction sur une seule observation
        pred  = clf.predict(pd.DataFrame(X_val_temp.iloc[i,:]).T)
        proba = clf.predict_proba(pd.DataFrame(X_val_temp.iloc[i,:]).T)[0][1] 
        date = X_val_temp.index[i]
        
        print(dico_model[name])
       
        # Stratégie
        print(pred)
        Cap1,Qty1 = strategy_long_only(dico_model[name]["Cap1"].values[i], dico_model[name]["Qty1"].values[i], data["VIX"].loc[date], pred)
        Cap2,Qty2,Short = strat_long_short(dico_model[name]["Cap2"].values[i], dico_model[name]["Qty2"].values[i], data["VIX"].loc[date], proba, dico_model[name]["Short"].values[i])
        Cap3,Qty3 = strategy_long_only_SP500(dico_model[name]["Cap3"].values[i], dico_model[name]["Qty3"].values[i], sp500.loc[date].values[0], proba)
        
        
        dict_s = {"Proba" : proba ,"Pred" :pred,"Cap1" : Cap1,"Cap2": Cap2,"Cap3":Cap3,"Qty1":Qty1,"Qty2":Qty2,"Qty3":Qty3, "Short" : Short}
        
        dico_model[name] = dico_model[name].append(pd.DataFrame(dict_s,index=[X_val.index[i]]))
        
        # Append une observation au Train 
        X_train.loc[date] = X_val.iloc[i,]
        y_train.loc[date] = y_train.iloc[i]
        

for model in dico_model.keys():
    print(dico_model[model])       

(253, 103) (85, 103) (38, 103) (253,) (85,) (38,)
 Results for RIDGE_AUC:
 Returned hyperparameter: {'solver': 'liblinear'}
 Best classification score in validation set is: 79.5
--------- 3.55 secondes ---------
-----------------------------------------------------
            Proba  Pred  Cap1  Cap2  Cap3  Qty1  Qty2  Qty3  Short
30/04/2012      0     0   100   100   100     0     0     0  False
[1]
 Results for RIDGE_Class:
 Returned hyperparameter: {'solver': 'lbfgs'}
 Best classification score in validation set is: 80.30000000000001
--------- 0.13 secondes ---------
-----------------------------------------------------
            Proba  Pred  Cap1  Cap2  Cap3  Qty1  Qty2  Qty3  Short
30/04/2012      0     0   100   100   100     0     0     0  False

C:\Users\ammyd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\ammyd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg


[1]
 Results for RIDGE_Dev:
 Returned hyperparameter: {'solver': 'lbfgs'}
 Best classification score in validation set is: 80.30000000000001
--------- 0.15 secondes ---------
-----------------------------------------------------
            Proba  Pred  Cap1  Cap2  Cap3  Qty1  Qty2  Qty3  Short
30/04/2012      0     0   100   100   100     0     0     0  False
[1]
 Results for LASSO_AUC:
 Returned hyperparameter: {'solver': 'liblinear'}
 Best classification score in validation set is: 80.30000000000001
--------- 0.06 secondes ---------
-----------------------------------------------------
            Proba  Pred  Cap1  Cap2  Cap3  Qty1  Qty2  Qty3  Short
30/04/2012      0     0   100   100   100     0     0     0  False
[1]
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] learning_rate is set=0.1, learning_rate= will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] Unknown parameter:

In [ ]:
dico_model["LIGHTGBM"].to_csv("LIGHTGBM_ACCuracy.csv")
